# Exploring and cleaning Barcelona data

There are several .csv files in the data/ folder. For this mid-term projecto we are going to use **bus_stops.csv** and **transports.csv** data.

The main idea is to plot al types of transportation methods in Barcelona and create a dashboard to get the nearests transportation locations given a point in barcelona and a radius

---

## Data exploration

In [1]:
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import os
from dotenv import load_dotenv
load_dotenv()

bus_stops = pd.read_csv("bus_stops.csv")
transports = pd.read_csv("transports.csv")

In [2]:
bus_stops.head()

,Code,Transport,Longitude,Latitude,Bus.Stop,District.Name,Neighborhood.Name
0,K014,Day bus stop,2.171619,41.413744,BUS -192--,Horta-Guinardó,el Guinardó
1,K014,Day bus stop,2.134902,41.420222,BUS -124--,Gràcia,Vallcarca i els Penitents
2,K014,Day bus stop,2.162913,41.423187,BUS -117--,Horta-Guinardó,la Font d'en Fargues
3,K014,Day bus stop,2.163667,41.422899,BUS -117--,Horta-Guinardó,la Font d'en Fargues
4,K014,Day bus stop,2.120212,41.397209,BUS -130--,Sarrià-Sant Gervasi,Sarrià


In [3]:
bus_stops[bus_stops['District.Name'].isnull()]

,Code,Transport,Longitude,Latitude,Bus.Stop,District.Name,Neighborhood.Name
215,K015,Night bus stop,2.175509,41.463715,NITBUS -N3--,NaN,NaN
269,K014,Day bus stop,2.175149,41.463839,BUS -104--,NaN,NaN
293,K014,Day bus stop,2.132912,41.359207,BUS -79-125--,NaN,NaN
443,K015,Night bus stop,2.209424,41.425163,NITBUS -N11-N2--,NaN,NaN
1204,K014,Day bus stop,2.055835,41.365599,BUS -78-L46-L52--,NaN,NaN
1232,K014,Day bus stop,2.209424,41.425163,BUS -B25-V33--,NaN,NaN
1236,K014,Day bus stop,2.124490,41.373531,BUS -LH2--,NaN,NaN
1378,K014,Day bus stop,2.175509,41.463715,BUS -62-76-104-83--,NaN,NaN
1427,K014,Day bus stop,2.114631,41.375908,BUS -54-157-D20-H8--,NaN,NaN
1428,K015,Night bus stop,2.114631,41.375908,NITBUS -N14-N3--,NaN,NaN


In [4]:
bus_stops.Code.unique()

array(['K014', 'K015', 'K016', 'K017'], dtype=object)

In [5]:
bus_stops.Transport.unique()

array(['Day bus stop', 'Night bus stop', 'Airport bus stop',
       'Bus station'], dtype=object)

In [6]:
transports.head()

,Code,Transport,Longitude,Latitude,Station,District.Name,Neighborhood.Name
0,K001,Underground,2.119370,41.399203,FGC (L6) - REINA ELISENDA (Sortida Duquesa d'O...,Sarrià-Sant Gervasi,Sarrià
1,K001,Underground,2.135427,41.397791,FGC (L6) - LA BONANOVA-,Sarrià-Sant Gervasi,Sant Gervasi - Galvany
2,K001,Underground,2.185391,41.451492,METRO (L11) - CASA DE L'AIGUA (C. Vila-Real)-,Nou Barris,la Trinitat Nova
3,K001,Underground,2.174473,41.460889,METRO (L11) - CIUTAT MERIDIANA (C. Pedraforca)-,Nou Barris,Ciutat Meridiana
4,K001,Underground,2.168588,41.387200,METRO (L1) - CATALUNYA (Rda. Universitat)-,Eixample,la Dreta de l'Eixample


In [7]:
transports.Code.unique()

array(['K001', 'K002', 'K011', 'K008', 'K003', 'K004', 'K010', 'K009'],
      dtype=object)

In [8]:
transports.Transport.unique()

array(['Underground', 'Railway (FGC)', 'Tram', 'Maritime station',
       'RENFE', 'Airport train', 'Cableway', 'Funicular'], dtype=object)

Below we can see that each unique value of *Code* is associated with a unique value of *Transport*, so now we now that we can either filter or look by any one of them

In [9]:
transports.groupby('Code').Transport.value_counts()

Code  Transport       
K001  Underground         463
K002  Railway (FGC)        54
K003  RENFE                33
K004  Airport train         9
K008  Maritime station     16
K009  Funicular             6
K010  Cableway              5
K011  Tram                 65
Name: Transport, dtype: int64

The first thing we notice is that the datasets don't have overlaping data. The **transports.csv** file contains all transportation methods in the city that are not bus stations.

Also, both data sets have the same number of columns and the columns contain the same type of information. The only difference is the name of the column *Station* in transports data that is called *Bus.Stop* on the other DF.

It would be nice to have only one dataset to work with, this is why I am going to concat the datasets, but first, I need to check if there is consistency in the data.

In [10]:
bus_stops.describe(include='all')

,Code,Transport,Longitude,Latitude,Bus.Stop,District.Name,Neighborhood.Name
count,3162,3162,3162.000000,3162.000000,3162,3146,3146
unique,4,4,NaN,NaN,711,10,73
top,K014,Day bus stop,NaN,NaN,NITBUS -N4--,Sarrià-Sant Gervasi,la Dreta de l'Eixample
freq,2324,2324,NaN,NaN,81,439,137
mean,NaN,NaN,2.159824,41.405002,NaN,NaN,NaN
std,NaN,NaN,0.027898,0.027150,NaN,NaN,NaN
min,NaN,NaN,2.055835,41.321643,NaN,NaN,NaN
25%,NaN,NaN,2.140627,41.386903,NaN,NaN,NaN
50%,NaN,NaN,2.161103,41.405734,NaN,NaN,NaN
75%,NaN,NaN,2.180406,41.423526,NaN,NaN,NaN


In [11]:
transports.describe(include='all')

,Code,Transport,Longitude,Latitude,Station,District.Name,Neighborhood.Name
count,651,651,651.000000,651.000000,651,487,487
unique,8,8,NaN,NaN,540,10,63
top,K001,Underground,NaN,NaN,METRO (L1) - BELLVITGE-,Eixample,la Dreta de l'Eixample
freq,463,463,NaN,NaN,5,120,61
mean,NaN,NaN,2.156517,41.399351,NaN,NaN,NaN
std,NaN,NaN,0.040061,0.028520,NaN,NaN,NaN
min,NaN,NaN,2.005280,41.304419,NaN,NaN,NaN
25%,NaN,NaN,2.135074,41.377156,NaN,NaN,NaN
50%,NaN,NaN,2.164163,41.396315,NaN,NaN,NaN
75%,NaN,NaN,2.182347,41.418754,NaN,NaN,NaN


The longitude and latitude in both datasets seem to bee inside the bounds of Barcelona. The column *District.Name* have the same number of unique values, we will have to check if they are the same. For the *Neighborhood.Name* column, they have different number of unique values, we will have to check if the 63 different neighborhoods in *transports* DF are in the 73 unique neighborhoods of *bus_stops* DF.

In [12]:
sorted(bus_stops['District.Name'].dropna().unique()) == sorted(transports['District.Name'].dropna().unique())

True

In [13]:
for neighborhood in transports['Neighborhood.Name'].dropna().unique():
    if neighborhood not in bus_stops['Neighborhood.Name'].dropna().unique():
        print(f'{neighborhood} not included')

Now we know that there is consistency between the datasets and we can concat them without problem or previous data cleaning. The only thing we still have to check is the missing values

In [14]:
bus_stops.isnull().sum() / bus_stops.shape[0]

Code                 0.00000
Transport            0.00000
Longitude            0.00000
Latitude             0.00000
Bus.Stop             0.00000
District.Name        0.00506
Neighborhood.Name    0.00506
dtype: float64

In [15]:
transports.isnull().sum() / transports.shape[0]

Code                 0.00000
Transport            0.00000
Longitude            0.00000
Latitude             0.00000
Station              0.00000
District.Name        0.25192
Neighborhood.Name    0.25192
dtype: float64

The DF with bus stops has ver few missing values, but transports dataframe has 25% NaN's in *District.Name* and *Neighborhood.Name*. It seem that all rows with NaN in one of the columns have also NaN in the other one.

In [16]:
# Check if when there is a NaN in District.Name there is also one in Neighborhood.Name
len(transports[transports['District.Name'].isnull() & transports['Neighborhood.Name'].isnull()]) == len(transports[transports['District.Name'].isnull()])

True

The missing values are not a problem because we have the latitude and longitude, thus, we can use those variables to plot the data in a map and use geoqueries to access those observations.

We are going to concat the data so that we can work with only one Dataframe. To do this, I will have to change the column name of *Bus.Stop* to *Station*. This will make things easier in the future.

In [17]:
# Rename column Bus.Stop
bus_stops.rename({'Bus.Stop': 'Station'}, inplace=True, axis='columns')

# Concat Dataframes
transport_data = pd.concat([bus_stops, transports])

In [18]:
transport_data.head()

,Code,Transport,Longitude,Latitude,Station,District.Name,Neighborhood.Name
0,K014,Day bus stop,2.171619,41.413744,BUS -192--,Horta-Guinardó,el Guinardó
1,K014,Day bus stop,2.134902,41.420222,BUS -124--,Gràcia,Vallcarca i els Penitents
2,K014,Day bus stop,2.162913,41.423187,BUS -117--,Horta-Guinardó,la Font d'en Fargues
3,K014,Day bus stop,2.163667,41.422899,BUS -117--,Horta-Guinardó,la Font d'en Fargues
4,K014,Day bus stop,2.120212,41.397209,BUS -130--,Sarrià-Sant Gervasi,Sarrià


In [19]:
transport_data.Transport.unique()

array(['Day bus stop', 'Night bus stop', 'Airport bus stop',
       'Bus station', 'Underground', 'Railway (FGC)', 'Tram',
       'Maritime station', 'RENFE', 'Airport train', 'Cableway',
       'Funicular'], dtype=object)

In [20]:
transport_data['Station'].value_counts()

NITBUS -N4--                             81
NITBUS -N3--                             71
NITBUS -N6--                             67
NITBUS -N8--                             61
NITBUS -N0--                             60
                                         ..
BUS -39-45-V17--                          1
BUS -47-132-82-D40-V29-D50--              1
BUS -H12-V31-136--                        1
BUS -23-109--                             1
METRO (L3) - POBLE SEC (C. de Manso)-     1
Name: Station, Length: 1251, dtype: int64

In [21]:
transports.head()

,Code,Transport,Longitude,Latitude,Station,District.Name,Neighborhood.Name
0,K001,Underground,2.119370,41.399203,FGC (L6) - REINA ELISENDA (Sortida Duquesa d'O...,Sarrià-Sant Gervasi,Sarrià
1,K001,Underground,2.135427,41.397791,FGC (L6) - LA BONANOVA-,Sarrià-Sant Gervasi,Sant Gervasi - Galvany
2,K001,Underground,2.185391,41.451492,METRO (L11) - CASA DE L'AIGUA (C. Vila-Real)-,Nou Barris,la Trinitat Nova
3,K001,Underground,2.174473,41.460889,METRO (L11) - CIUTAT MERIDIANA (C. Pedraforca)-,Nou Barris,Ciutat Meridiana
4,K001,Underground,2.168588,41.387200,METRO (L1) - CATALUNYA (Rda. Universitat)-,Eixample,la Dreta de l'Eixample


In [22]:
transports.groupby('Code').Station.value_counts()

Code  Station                                                       
K001  METRO (L1) - BELLVITGE-                                           5
      METRO (L1) - PLAÇA DE SANTS (C. de Sants)-                        4
      METRO (L1) - RAMBLA JUST OLIVERAS-                                4
      METRO (L1) - SANTA COLOMA-                                        4
      METRO (L2) - PEP VENTURA-                                         4
                                                                       ..
K011  TRAMVIA BLAU - AV. TIBIDABO (Rda. de Dalt-Bosch i Alsina)-        1
      TRAMVIA BLAU - AV. TIBIDABO (Romà Macaya-Dr. Ribas Perdigó)-      1
      TRAMVIA BLAU - AV. TIBIDABO (Teodor Roviralta-Adrià Margarit)-    1
      TRAMVIA BLAU - PL. DR. ANDREU-                                    1
      Tramvia (T3) - SANT FELIU/CONSELL COMARCAL-                       1
Name: Station, Length: 540, dtype: int64

I want to clean the data so that for each station and type of transport I have the lines that pass through. I will create a new column named *Lines* to stores al the lines that pass through each station with lists. There are some type of transports which don't have lines, for example RENFE, these types of transport will have their transport type as the line name, so that I don't have blank lists. There is also a special case for the tramvias. There is one type of tramvia called *TRAMVIA BLAU* which doesn't have a line number, for the stations where this tramvia passes, the lists will contain *BLAU*.

In [23]:
# Get the lines of each station

def get_lines(row):
    if row['Code'] in ['K002', 'K003', 'K004', 'K008', 'K009', 'K010', 'K017']:
        if row['Code'] == 'K002':
            return 'FGC'
        else:
            return row['Transport']
    elif row['Code'] == 'K011' and 'blau' in row['Station'].lower():
        return 'BLAU'
    else:
        res = re.findall(r'\w*\d+', row['Station']) #Get the line. The line can be consecutive digits or a letter followed by one or more digits
        if res  == []:
            if '-BCTE-' in row['Station']:
                res.append('BCTE')
            if '-BCTO-' in row['Station']:
                res.append('BCTO')
            if '-CJ-' in row['Station']:
                res.append('CJ')
        return res


In [24]:
# Create new column Lines

transport_data['Lines'] = transport_data.apply(get_lines, axis=1)
transport_data.head()

,Code,Transport,Longitude,Latitude,Station,District.Name,Neighborhood.Name,Lines
0,K014,Day bus stop,2.171619,41.413744,BUS -192--,Horta-Guinardó,el Guinardó,[192]
1,K014,Day bus stop,2.134902,41.420222,BUS -124--,Gràcia,Vallcarca i els Penitents,[124]
2,K014,Day bus stop,2.162913,41.423187,BUS -117--,Horta-Guinardó,la Font d'en Fargues,[117]
3,K014,Day bus stop,2.163667,41.422899,BUS -117--,Horta-Guinardó,la Font d'en Fargues,[117]
4,K014,Day bus stop,2.120212,41.397209,BUS -130--,Sarrià-Sant Gervasi,Sarrià,[130]


---

To conclude with the cleaning, we are going to export the data to a csv file so that we can work with the clean data.

In [25]:
# Export data to csv
# Uncoment the line below if you want to generate the clean data
#transport_data.to_csv("transports_clean.csv")

---

The next thing that we need to do is to have the data in geoJSON format so that we can make geoqueries. To do this, we have created a collection in mongodb that will be filled with the new data in the geoJSON format. 

The cleaned data has been added as a collection to mongodb. We are goin to connect to mongo, get that database in JSON format, transform the data to geoJSON format and insert that new data to the blank collection that we have created called geo_transports

In [26]:
url = os.getenv('url')
client = MongoClient(url)
BCN = client.get_database('BCN')

In [27]:
transports_json = BCN.transports_clean

In [28]:
# Erase _id, and get the data in a list to iterate over it and create the new variable Location

geo_transports = list(transports_json.find({},{"_id":0}))

In [29]:
geo_transports[0]

{'': '0',
 'Code': 'K014',
 'Transport': 'Day bus stop',
 'Longitude': '2.171619',
 'Latitude': '41.413744',
 'Station': 'BUS -192--',
 'District': {'Name': 'Horta-Guinardó'},
 'Neighborhood': {'Name': 'el Guinardó'},
 'Lines': "['192']"}

In [30]:
# Now for each document in the data, we are going to transform it to geoJSON format. The variable data will later be used to fill the blank collection that we have in mongo.

data = []
i = 0
for gt in geo_transports:
    gt["Location"] = {
        "type":"Point",
        "coordinates": [gt["Longitude"], gt["Latitude"]]
    }
    del gt['Lines']
    gt['Lines'] = transport_data.iloc[i].loc['Lines']
    data.append(gt)
    i += 1

In [31]:
data[73]

{'': '73',
 'Code': 'K014',
 'Transport': 'Day bus stop',
 'Longitude': '2.192252',
 'Latitude': '41.379159',
 'Station': 'BUS -BCTE--',
 'District': {'Name': 'Ciutat Vella'},
 'Neighborhood': {'Name': 'la Barceloneta'},
 'Location': {'type': 'Point', 'coordinates': ['2.192252', '41.379159']},
 'Lines': ['BCTE']}

In [32]:
# Now we get the blanck collection and fill it with the data
new_transports = BCN.geo_transports

new_transports.insert_many(data)